In [ ]:
!pip install geopandas

In [ ]:
!pip install shapely

In [ ]:
!pip install rtree 

!pip install pygeos 

!pip install folium

!pip install fiona
!pip install pyproj
!pip install psycopg2 
!pip install GeoAlchemy2
!pip install geopy

     |████████████████████████████████| 994 kB 4.1 MB/s 


In [ ]:
!pip install pandas fiona shapely pyproj rtree

In [ ]:
import math
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

import folium
from folium import Marker
from folium.plugins import MarkerCluster

In [ ]:
starbucks = pd.read_csv("/content/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [ ]:
starbucks.shape

(2821, 6)

In [ ]:
print(starbucks.isnull().sum())

rows_with_missing = starbucks[starbucks['City'] == "Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [ ]:
def meu_geodecodificador(row):
  point = geocode(row, provider="nominatim").geometry[0]
  return pd.Series({'Longitude':point.x, 'Latitude':point.y})

berkeley_locais = rows_with_missing.apply(lambda x: meu_geodecodificador(x['Address']), axis=1)
starbucks.update(berkeley_locais)

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [ ]:
mapa = folium.Map(location=[37.88, -122.26], zoom_start=13)

for idx, row in starbucks[starbucks["City"] == 'Berkeley'].iterrows():
  Marker([row['Latitude'], row['Longitude']]).add_to(mapa)

mapa


In [ ]:
ca_paises = gpd.read_file("/content/CA_county_boundaries.shp")
ca_paises.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


In [ ]:
ca_paises.shape

(58, 4)

In [ ]:
ca_pop = pd.read_csv("/content/CA_county_population.csv", index_col="GEOID")
ca_pop.head(1)

,population
GEOID,
6001,1666753


In [ ]:
ca_high_earners = pd.read_csv("/content/CA_county_high_earners.csv", index_col='GEOID')
ca_high_earners.head(1)

,high_earners
GEOID,
6001,145696


In [ ]:
ca_median_age = pd.read_csv("/content/CA_county_median_age.csv", index_col="GEOID")
ca_median_age.head(1)

,median_age
GEOID,
6001,37.3


In [ ]:
cols_to_add = ca_pop.join([ca_high_earners, ca_median_age]).reset_index()
ca_stats = ca_paises.merge(cols_to_add, on="GEOID")

In [ ]:
ca_stats['density'] = ca_stats['population']/ca_stats['area_sqkm']
ca_stats.head(1)

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638


In [ ]:
sel_counties = ca_stats[((ca_stats.high_earners > 100000) &
                         (ca_stats.median_age < 38.5) &
                         (ca_stats.density > 285) &
                         ((ca_stats.median_age < 35.5) |
                         (ca_stats.density > 1400) |
                         (ca_stats.high_earners > 500000)))]

In [ ]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
locais_interesse = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locais_interesse)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  """Entry point for launching an IPython kernel.


ImportError: ignored